# 03. Python을 활용한 텍스트 마이닝 8.텍스트 분석-감성 분석(Sentiment Analysis)

* 싸이그래머 / 어바웃 파이썬
* 곽대기

Sentiment Analysis(SA) 에는 크게 세 가지 방식이 있습니다.

* knowledge-based techniques
* statistical methods
* hybrid approaches

먼저 knowledge-based technique은 단어들의 감정의 정도를 평가하는 사전을 만들고 이를 활용해서 단어 또는 글의 감정 상태를 평가하는 방법입니다.

예를 들어 SentiWordnet은 단어 마다의 긍/부정 척도를 더한 대표적인 db로 Python의 nltk에 포함되어 있습니다.


다음으로 statistical method는 SVM(support vector machine)이나 ‘bag of words’등 다양한 ML 방법론을 활용해서 단어의 감정을 분류하는 방법입니다. Python의 nltk 패키지에 naive bayes classifier도 통계적 방법 중 하나입니다.


마지막으로 hybrid approaches의 경우 두 방법을 적절하게 혼합하여 활용하는 방법입니다.

SA에서 가장 힘든 부분 중 하나는 비정형인 자연어의 노이즈들이 많다는 점입니다. 인간의 경우 다양한 문맥에 따라서 흐름을 쉽게 파악할 수 있지만 기계는 어렵습니다.

예를 들어 ‘내가 치킨을 싫어하는 것은 아니야.’ 라는 문장을 보았을 때 사람들은 이 문장이 postive 또는  neutral 하다고 쉽게 판단하지만 긍/부정 단어에 점수를 매기는 knowledge-based technique에서는 negative 문장으로 해석될 수 있습니다. 이처럼 이중 부정문을 다루거나 비속어, 은유, 비유 또는 풍자 등과 같은 skill을 학습하기 힘들기 때문에 현재까지는 한계점이 뚜렸하다고 생각합니다.



![clementine.jpg](attachment:clementine.jpg)
![clementine.png](attachment:clementine.png)

### 데이터 셋 준비

http://ai.stanford.edu/~amaas/data/sentiment/

에서 데이터셋을 받습니다. 이 데이터는 IMDb라는 미국 영화 사이트의 리뷰를 50,000개 수집하여 리뷰마다 각각 긍/부정 분류로 나눈 데이터 셋입니다.

In [1]:
import os
files = os.listdir('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos')
 
first_file = files[0]
with open('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos/{}'.format(first_file),'r',encoding='utf-8') as f:
    review = f.read()
    f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos'

In [ ]:
pos_train_list=[]
for file in files:
    with open('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos/{}'.format(file),'r',encoding='utf-8') as f:
        review = f.read()
        f.close()
    pos_train_list.append(review)
print(len(pos_train_list))

In [2]:
import nltk
from nltk.corpus import sentiwordnet as swn

list(swn.senti_synsets('hate'))

LookupError: 
**********************************************************************
  Resource 'corpora/sentiwordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/home/dex/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [ ]:
list(swn.senti_synsets('hate','v'))[0].pos_score()
list(swn.senti_synsets('hate','v'))[0].neg_score()

In [3]:
def word_sentiment_calculator(word,tag):
    pos_score=0
    neg_score=0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word,'n')))>0:
        syn_set = list(swn.senti_synsets(word,'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word,'v')))>0:
        syn_set = list(swn.senti_synsets(word,'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word,'a')))>0:
        syn_set = list(swn.senti_synsets(word,'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word,'r')))>0:
        syn_set = list(swn.senti_synsets(word,'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set),neg_score/len(syn_set))

In [4]:
word_sentiment_calculator('love','VB')

LookupError: 
**********************************************************************
  Resource 'corpora/sentiwordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/home/dex/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [5]:
#문장에서도 사용가능

def sentence_sentiment_calculator(pos_tags):
    pos_score=0
    neg_score=0
    for word,tag in pos_tags:
        pos_score += word_sentiment_calculator(word,tag)[0]
        neg_score += word_sentiment_calculator(word,tag)[1]
    return(pos_score,neg_score)

In [ ]:
#메인 

pos_files = os.listdir('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos/')[:10]
neg_files = os.listdir('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/neg/')[:10]

actual = [1]*10 + [0]*10
predicted=[]

def sentence_sentiment_calculator2(pos_tags):
    pos_score=0
    neg_score=0
    s_tk=nltk.word_tokenize(pos_tags)
    pos_tags=nltk.pos_tag(s_tk)
    for word,tag in pos_tags:
        pos_score += word_sentiment_calculator(word,tag)[0]
        neg_score += word_sentiment_calculator(word,tag)[1]
    return(pos_score,neg_score)

for file in pos_files:
    with open('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/pos/{}'.format(file),'r',encoding='utf-8') as f:
        scores = sentence_sentiment_calculator2(f.read())
        
        if scores[0] >= scores[1]:
            predicted.append(1)
        else:
            predicted.append(0)
        f.close()
        
for file in neg_files:
    with open('aclImdb_v1.tar/aclImdb_v1/aclImdb/train/neg/{}'.format(file),'r',encoding='utf-8') as f:
        scores = sentence_sentiment_calculator2(f.read())
        
        if scores[0] >= scores[1]:
            predicted.append(1)
        else:
            predicted.append(0)
        f.close()
        
correct = 0
incorrect = 0
for i in range(20):
    if actual[i] == predicted[i]:
        correct +=1
    else:
        incorrect +=1
        
print(actual)
print(predicted)

print('Number of correct instance: ',correct)
print('Number of incorrect instance: ',incorrect)